## Configuration and authentication demo

This notebook demonstrates how to generate your config file. The most complicated part of this is generating a token that can be passed to the ADAM REST server for access to API methods.

In [ ]:
import adam
from adam import Auth
from adam import ConfigManager
from adam import RestRequests

import webbrowser
import urllib
import os
from shutil import copyfile

Attempts to read config that is already set up from a config.json file in your working directory. Feel free to point it somewhere else. If it fails to read that file, it will copy the template from adam.

In [ ]:
config_file = os.getcwd() + '/config.json'
try:
    f = open(config_file)
    f.close()
except:
    copyfile(os.path.dirname(adam.__file__) + '/config_template.json', config_file)

config_manager = ConfigManager(config_file)
config = config_manager.get_config()

# Feel free to run again with the following line to fill in your dev token.
# config = config_manager.get_config(environment='dev')

Now attempt to authenticate with the token in the config. If that fails, it tries to reauthenticate. This is a flow that directs a user to a page where they can log in and get a token. If that new token works, this will update the config file so the token can be used next time.

In [ ]:
url = config.get_url()
rest = RestRequests(url)
auth = Auth(rest)

In [ ]:
token = config.get_token()

try:
    auth.authenticate(token)
    if auth.get_logged_in():
        print('Welcome, ' + auth.get_user())
    else:
        input("A website will pop up and you will be given options to log in to "
              "retrieve a token. Please copy the token and return here. Press enter to continue...")
        o = urllib.parse.urlparse(url)
        token_url = "%s://%s/%s" % (o.scheme, o.netloc, 'token.html')  # Yes, seriously. This is how it's done.
        webbrowser.open(token_url)
        print(token_url)
        
        token = input("Token: ")
        print('\n')
        if auth.authenticate(token):
            config.set_token(auth.get_token())
            config_manager.to_file(config_file)
            print('Updated config file with token for ' + url)
            print('Welcome, ' + auth.get_user())
        else:
            print('Could not authenticate user.')
except RuntimeError as e:
    print('Encountered server error while attempting to authenticate: ' + str(e))

If authentication was successful, the token can now be used for any API method requiring authentication.

Now an easy extra part. Do you have a workspace project? Enter its uuid here. It will be used as a parent of any temporary working projects created.

In [ ]:
# If you need some help finding the uuid, try the following (it might take a few seconds):
# from adam import Projects
# from adam import AuthenticatingRestProxy
# Projects(AuthenticatingRestProxy(RestRequests(config.get_url()), token)).print_projects()

workspace = input('Workspace project UUID: ')
config.set_workspace(workspace)
config_manager.to_file(config_file)